# Import and Init

# 1.1 Python and Stuffs

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Définir le backend souhaité (par exemple, 'tensorflow' ou 'theano')
backend = 'torch'  # Remplacez 'tensorflow' par le backend de votre choix

# Définir la variable d'environnement KERAS_BACKEND
os.environ['KERAS_BACKEND'] = backend

# Importer Keras après avoir défini la variable d'environnement


import PIL
from PIL import Image

In [51]:
import keras


In [20]:
import h5py

# 1.2 Parameters

In [21]:
# filesystem
Data_dir="/home/onyxia/work/AutoEncoder/Data"
Codes_dir="/home/onyxia/work/AutoEncoder/Codes"
Outputs_dir="/home/onyxia/work/AutoEncoder/Outputs"

# Preprocessing 

# Model

2 Load  Data

In [22]:
def retrieve_data(filename) :
    with h5py.File(Data_dir+"/"+filename, 'r') as f:
    # Créer un dataset
       x_train=  f['x_train'][:] 
       y_train=  f['y_train'][:] 
       x_test=  f['x_test'][:] 
       y_test=  f['y_test'][:] 

    return x_train,y_train,x_test,y_test

In [23]:
x_train,y_train,x_test,y_test=retrieve_data("dataset_Images.h5")

In [24]:
x_train.shape

(8000, 28, 28)

3 BUILD MODEL

3.1 ENCODER Model


In [25]:
shape= input_shape+tuple([1])
shape

(0, 5, 1)

In [76]:
def simple_encoder(input_shape,cl,dl,latent_dim,activation="relu",name="encoder",strides=2,padding="same"):
    n_filter=cl['n_filter']
    kernel_size=cl['kernel_size']
    dl_size=dl['dl_size']
    dl_activation=dl["activation"]
    
    inputs=keras.Input(shape= input_shape+tuple([1]),name="Input_Layers")
    
     # add Convolution Layers
    x=keras.layers.Conv2D(n_filter,kernel_size,strides=strides,padding=padding,name="Conv2D_1")(inputs)
    x=keras.layers.Conv2D(n_filter,kernel_size,strides=strides,padding=padding,name="Conv2D_2")(x)
    x=keras.layers.Conv2D(n_filter,kernel_size,strides=strides,padding=padding,name="Conv2D_3")(x)
     # add Flatten layers
    x=keras.layers.Flatten(name="Flatten")(x)
    
     # add Dense Layers
    x=keras.layers.Dense(dl_size,activation=dl_activation,name="Dense_1")(x)
    x=keras.layers.Dense(int(dl_size/2),activation=dl_activation,name="Dense_2")(x)
    x=keras.layers.Dense(int(dl_size/4),activation=dl_activation,name="Dense_3")(x)



    ouputs_Encoder=keras.layers.Dense(latent_dim,activation=activation,name="Dense_output")(x)

    # define model

    encoder=keras.models.Model(inputs,ouputs_Encoder,name="Encoder")

    return encoder

In [77]:
def conv_layer_params(n_filter=32,kernel_size=(3,3)):
    cl={}
    cl["n_filter"]=n_filter
    cl["kernel_size"]=kernel_size
    return cl


In [94]:
def dense_layer_params(dl_size=100,activation="relu"):
    dl={}
    dl["dl_size"]=dl_size
    dl["activation"]=activation
    return dl

In [79]:
cl=conv_layer_params()
dl=dense_layer_params()
input_shape=(28,28)
latent_dim=10
encoder=simple_encoder(input_shape,cl,dl,latent_dim,activation="relu",name="encoder",strides=2,padding="same")

In [80]:
encoder.summary()

Model: "Encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ Input_Layers (InputLayer)       │ (None, 28, 28, 1)         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Conv2D_1 (Conv2D)               │ (None, 14, 14, 32)        │        320 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Conv2D_2 (Conv2D)               │ (None, 7, 7, 32)          │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Conv2D_3 (Conv2D)               │ (None, 4, 4, 32)          │      9,248 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Flatten (Flatten)               │ (None, 512)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense_1 (Dense)                 │ (None, 100)               │     51,300 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense_2 (Dense)                 │ (None, 50)                │      5,050 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense_3 (Dense)                 │ (None, 25)                │      1,275 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense_output (Dense)            │ (None, 10)                │        260 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 76,701 (299.61 KB)

 Trainable params: 76,701 (299.61 KB)

 Non-trainable params: 0 (0.00 B)

 Ajout fonction qui definit la partie décoder

In [170]:
def decoder(input_len,dl1,Tcl1,padding="same",strides=2,output_decoder_shape=(28,28,1)):
   dl1_len=dl1["dl_size"]
   dl1_activation=dl1["activation"]
   dl2_len=int(dl1_len*2)
   dl3_len=49*int((dl2_len*2)/49) # je multiplie et je divise par 49 pour pourvoir à la suite extraire un reshape de la forme (7,7,x)tp 7*7*x=dl3_len
   RshapeL_shape=(7,7,int(dl3_len/49))
   Tcl1_len=Tcl1["n_dilator"] #kernel_size
   Tcl1_kernel_size=Tcl1["kernel_size"]  # activation
   Tcl1_activation=Tcl1["activation"]

# Add inputs layers
   inputs=keras.Input((latent_dim,),name="input_decoder")
# add Dense
   x=keras.layers.Dense(dl1_len,activation=dl1_activation,name="Dense1")(inputs)
   x=keras.layers.Dense(dl2_len,activation=dl1_activation,name="Dense2")(x)
   x=keras.layers.Dense(dl3_len,activation=dl1_activation,name="Dense3")(x)

# Reshape Layers
   x=keras.layers.Reshape(RshapeL_shape)(x)


# Add Transposed Convolutions layers
   x=keras.layers.Conv2DTranspose(Tcl1_len,Tcl1_kernel_size,activation=Tcl1_activation,padding=padding,strides=strides,name="TC1")(x)
   x=keras.layers.Conv2DTranspose(Tcl1_len*2,kernel_size=Tcl1_kernel_size,activation=Tcl1_activation,padding=padding,strides=int(strides*2),name="TC2")(x)
   #x=keras.layers.Conv2DTranspose(Tcl1_len*4,kernel_size=Tcl1_kernel_size,activation=Tcl1_activation,padding=padding,strides=strides,name="TC3")(x)

# add output decoder layers
   outputs_decoder=keras.layers.Conv2DTranspose(output_decoder_shape[-1],kernel_size=(3,3),activation=Tcl1_activation,padding="valid",strides=int(strides),name="TC_output")(x)
  ## Add output decoder layers
   #x = keras.layers.Conv2DTranspose(1, (3, 3), activation=Tcl1_activation, padding=padding, strides=strides,name="TC3")(x)

# Add Dense Layer for output
   #x = keras.layers.Dense(28 * 28 * 1, activation=Tcl1_activation,name="Dense4")(x)  # Adjust activation function as needed

# Reshape the output
   #outputs_decoder = keras.layers.Reshape((28, 28, 1),name='Reshape2')(x)


# get decoder model
   decoder=keras.models.Model(inputs,outputs_decoder,name="decoder")

   return decoder

In [136]:
def Tconv_layer_params(n_dilator=32,kernel_size=(3,3),activation="relu"):
    Tcl={}
    Tcl["n_dilator"]=n_dilator
    Tcl["kernel_size"]=kernel_size
    Tcl["activation"]=activation
    return Tcl

In [89]:
Tcl1=Tconv_layer_params()
Tcl1


{'n_dilator': 32, 'kernel_size': (3, 3), 'activation': 'relu'}

In [97]:
dl1=dense_layer_params()
dl1

{'dl_size': 100, 'activation': 'relu'}

In [173]:
import keras
from keras import layers

def decoder(input_len, dl1, Tcl1, padding="same", strides=2, output_decoder_shape=(28, 28, 1)):
    dl1_len = dl1["dl_size"]
    dl1_activation = dl1["activation"]
    dl2_len = int(dl1_len * 2)
    dl3_len = 49 * int((dl2_len * 2) / 49)  # Ajustement pour obtenir une taille qui peut être redimensionnée en (7, 7, x)
    RshapeL_shape = (7, 7, int(dl3_len / 49))
    Tcl1_len = Tcl1["n_dilator"]  # Taille du kernel
    Tcl1_kernel_size = Tcl1["kernel_size"]  # Activation
    Tcl1_activation = Tcl1["activation"]

    # Couche d'entrée
    inputs = keras.Input((input_len,), name="input_decoder")

    # Couches Denses
    x = layers.Dense(dl1_len, activation=dl1_activation, name="Dense1")(inputs)
    x = layers.Dense(dl2_len, activation=dl1_activation, name="Dense2")(x)
    x = layers.Dense(dl3_len, activation=dl1_activation, name="Dense3")(x)

    # Reshape
    x = layers.Reshape(RshapeL_shape)(x)

    # Couches de Convolution Transposée
    x = layers.Conv2DTranspose(Tcl1_len, Tcl1_kernel_size, activation=Tcl1_activation, padding=padding, strides=strides,
                                name="TC1")(x)
    x = layers.Conv2DTranspose(Tcl1_len * 2, kernel_size=Tcl1_kernel_size, activation=Tcl1_activation, padding=padding,
                                strides=strides, name="TC2")(x)

    # Couche de sortie
    outputs_decoder = layers.Conv2DTranspose(output_decoder_shape[-1], kernel_size=(3, 3), activation=Tcl1_activation,
                                              padding="same", strides=strides, name="TC_output")(x)

    # Création du modèle
    decoder = keras.models.Model(inputs, outputs_decoder, name="decoder")

    return decoder


In [174]:
input_len=10
decoder=decoder(input_len,dl1,Tcl1,padding="same",strides=2,output_decoder_shape=(28,28,1))


In [175]:
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_decoder (InputLayer)      │ (None, 10)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense1 (Dense)                  │ (None, 100)               │      1,100 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense2 (Dense)                  │ (None, 200)               │     20,200 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ Dense3 (Dense)                  │ (None, 392)               │     78,792 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ reshape_28 (Reshape)            │ (None, 7, 7, 8)           │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ TC1 (Conv2DTranspose)           │ (None, 14, 14, 32)        │      2,336 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ TC2 (Conv2DTranspose)           │ (None, 28, 28, 64)        │     18,496 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ TC_output (Conv2DTranspose)     │ (None, 56, 56, 1)         │        577 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 121,501 (474.61 KB)

 Trainable params: 121,501 (474.61 KB)

 Non-trainable params: 0 (0.00 B)